In [1]:
from __future__ import print_function

import os
import numpy as np
import nibabel as nib
from scipy.signal import medfilt
from multiprocessing import Pool, cpu_count

In [2]:
def create_dir(path):
    if not os.path.isdir(path):
        os.makedirs(path)


def load_nii(path):
    nii = nib.load(path)
    return nii.get_data(), nii.get_affine()


def save_nii(data, path, affine):
    nib.save(nib.Nifti1Image(data, affine), path)
    return


def denoise(volume, kernel_size=3):
    return medfilt(volume, kernel_size)


def rescale_intensity(volume, percentils=[0.5, 99.5], bins_num=256):
    obj_volume = volume[np.where(volume > 0)]
    min_value = np.percentile(obj_volume, percentils[0])
    max_value = np.percentile(obj_volume, percentils[1])

    if bins_num == 0:
        obj_volume = (obj_volume - min_value) / (max_value - min_value).astype(np.float32)
    else:
        obj_volume = np.round((obj_volume - min_value) / (max_value - min_value) * (bins_num - 1))
        obj_volume[np.where(obj_volume < 1)] = 1
        obj_volume[np.where(obj_volume > (bins_num - 1))] = bins_num - 1

    volume = volume.astype(obj_volume.dtype)
    volume[np.where(volume > 0)] = obj_volume

    return volume


def equalize_hist(volume, bins_num=256):
    obj_volume = volume[np.where(volume > 0)]
    hist, bins = np.histogram(obj_volume, bins_num, normed=True)
    cdf = hist.cumsum()
    cdf = (bins_num - 1) * cdf / cdf[-1]

    obj_volume = np.round(np.interp(obj_volume, bins[:-1], cdf)).astype(obj_volume.dtype)
    volume[np.where(volume > 0)] = obj_volume
    return volume


def unwarp_enhance(arg, **kwarg):
    return enhance(*arg, **kwarg)


def enhance(src_path, dst_path, kernel_size=3,
            percentils=[0.5, 99.5], bins_num=256, eh=True):
    print("Preprocess on: ", src_path)
    try:
        volume, affine = load_nii(src_path)
        volume = denoise(volume, kernel_size)
        volume = rescale_intensity(volume, percentils, bins_num)
        if eh:
            volume = equalize_hist(volume, bins_num)
        save_nii(volume, dst_path, affine)
    except RuntimeError:
        print("\tFailed on: ", src_path)

In [3]:
parent_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(parent_dir, "final_fyp_prep/data")
data_src_dir = os.path.join(data_dir, "2-Brain")
data_dst_dir = os.path.join(data_dir, "4-Enhance")
data_labels = ["healthy", "pd"]
create_dir(data_dst_dir)

In [4]:
data_src_paths, data_dst_paths = [], []
for label in data_labels:
    src_label_dir = os.path.join(data_src_dir, label)
    dst_label_dir = os.path.join(data_dst_dir, label)
    create_dir(dst_label_dir)
    for subject in os.listdir(src_label_dir):
        data_src_paths.append(os.path.join(src_label_dir, subject))
        data_dst_paths.append(os.path.join(dst_label_dir, subject))

In [5]:
kernel_size = 3
percentils = [0.5, 99.5]
bins_num = 256
eh = True

In [ ]:
# Test
# enhance(data_src_paths[0], data_dst_paths[0],
#         kernel_size, percentils, bins_num, eh)

# Multi-processing
subj_num = len(data_src_paths)
paras = zip(data_src_paths, data_dst_paths,
            [kernel_size] * subj_num,
            [percentils] * subj_num,
            [bins_num] * subj_num,
            [eh] * subj_num)
pool = Pool(processes=cpu_count())
pool.map(unwarp_enhance, paras)

Preprocess on:  /home/abdul/Desktop/final_fyp_prep/data/2-Brain/healthy/PPMI_103183_MR_3D_T2_FLAIR__br_raw_20211001010943238_92_S1068406_I1498877.nii.gz
Preprocess on:  /home/abdul/Desktop/final_fyp_prep/data/2-Brain/pd/PPMI_100005_MR_3D_T2_FLAIR__br_raw_20220224203326919_154_S1108072_I1548624.nii.gz
Preprocess on:  /home/abdul/Desktop/final_fyp_prep/data/2-Brain/healthy/PPMI_130190_MR_3D_T2_FLAIR__br_raw_20220225014525621_15_S1108182_I1548734.nii.gz
Preprocess on:  /home/abdul/Desktop/final_fyp_prep/data/2-Brain/pd/PPMI_102321_MR_3D_T2_FLAIR__br_raw_20211214130707070_191_S1090364_I1526455.nii.gz


<ipython-input-2-629f06e365a1>:8: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  return nii.get_data(), nii.get_affine()
<ipython-input-2-629f06e365a1>:8: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  return nii.get_data(), nii.get_affine()
<ipython-input-2-629f06e365a1>:8: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* d

Preprocess on:  /home/abdul/Desktop/final_fyp_prep/data/2-Brain/healthy/PPMI_101195_MR_3D_T2_FLAIR__br_raw_20210924164712654_79_S1066287_I1496369.nii.gz
Preprocess on:  /home/abdul/Desktop/final_fyp_prep/data/2-Brain/pd/PPMI_101742_MR_3D_T2_FLAIR__br_raw_20210914010302108_161_S1062111_I1491290.nii.gz
Preprocess on:  /home/abdul/Desktop/final_fyp_prep/data/2-Brain/healthy/PPMI_106127_MR_3D_T2_SAG_FLAIR__br_raw_20211124065644926_22_S1084533_I1519067.nii.gz
Preprocess on:  /home/abdul/Desktop/final_fyp_prep/data/2-Brain/pd/PPMI_101384_MR_3D_T2_FLAIR_SAGITAL__br_raw_20220105145950228_100_S1093335_I1530287.nii.gz
Preprocess on:  /home/abdul/Desktop/final_fyp_prep/data/2-Brain/healthy/PPMI_102447_MR_3D_T2_FLAIR__br_raw_20211001001431608_140_S1068380_I1498851.nii.gz
Preprocess on:  /home/abdul/Desktop/final_fyp_prep/data/2-Brain/healthy/PPMI_106127_MR_3D_T2_SAG_FLAIR_ND_br_raw_20211124065639325_19_S1084532_I1519066.nii.gz
Preprocess on:  /home/abdul/Desktop/final_fyp_prep/data/2-Brain/pd/PPMI